In [ ]:
import janitor
import pandas as pd
from IPython.display import display

# from pandasgui import show

- https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html
- https://pandas.pydata.org/docs/reference/api/pandas.option_context.html
- http://geoportal.cm-amadora.pt/graffiti/Arte_Urbana/index.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html
- https://stackoverflow.com/questions/39903090/efficiently-replace-values-from-a-column-to-another-column-pandas-dataframe
- https://pyjanitor-devs.github.io/pyjanitor/api/functions/
- https://ciberduvidas.iscte-iul.pt/consultorio/perguntas/o-uso-das-aspas--e-/13051
- https://justica.gov.pt/sdj/Normas-dos-servicos-digitais/Criacao-de-Conteudos/Livro-de-Estilo/Regras-e-coerencia#Respeitaralgumasregrasquandousamosaspas

In [ ]:
RAW_DATA = "./arte_urbana.xlsx"
OUTPUT_DATA = "../src/data.json"

In [ ]:
df = pd.read_excel(
    RAW_DATA,
    # index_col="ID",
    verbose=True,
    dtype={"Data": "Int64"},
)

In [ ]:
# show(df)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
df["name"].unique()

In [ ]:
df["Autoria"].unique()

In [ ]:
df["Freguesia"].unique()

In [ ]:
df["Tipologia"].unique()

In [ ]:
df["Localizacao"].unique()

In [ ]:
with pd.option_context("display.max_rows", None):
    display(df["Autoria"].value_counts(dropna=False))

In [ ]:
with pd.option_context("display.max_rows", None):
    display(df["Data"].value_counts(dropna=False))

In [ ]:
with pd.option_context("display.max_rows", None):
    display(df["Tipologia"].value_counts(dropna=False))

In [ ]:
# https://www.cm-amadora.pt/municipio/juntas-de-freguesia.html
# https://pt.wikipedia.org/wiki/Amadora#Divis%C3%A3o_administrativa_-_Freguesias
# 5/6 (no Alfragide)

with pd.option_context("display.max_rows", None):
    display(df["Freguesia"].value_counts(dropna=False))

In [ ]:
df.loc[df["Data"].isna(), "Descricao"].to_list()

In [ ]:
df["Descricao"].head()

In [ ]:
with pd.option_context("display.max_colwidth", None):
    display(df.query("Autoria == 'Vários'"))

In [ ]:
# APAURB: Associação Portuguesa de Arte Urbana (Portuguese Association of Urban Art)

df[df["Autoria"].str.contains("APAURB")].head()

In [ ]:
# cols = ["name", "Autoria", "Descricao"]
cols = ["Autoria", "Data", "Descricao"]

with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
    display(df[cols])

## Data Processing

In [ ]:
df = df.clean_names()
# df.columns

In [ ]:
df["autoria"] = df["autoria"].str.replace("\s/\s", "/", regex=True)

In [ ]:
# https://regex101.com/

varios_condition = df["autoria"] == "Vários"
# varios_value = df["descricao"].str.split("(").str[0]
varios_value = df["descricao"].str.extract(r"^(.+)\s\(\d{4}\)\.", expand=False)

apaurb_condition = df["autoria"] == "APAURB (artistas:Slap, Uber, Kurtz, Ephan e Dome)"
apaurb_value = df["autoria"].str.extract(r"\(artistas:(.+)\)", expand=False)

default_value = pd.NA

df = df.case_when(
    varios_condition,
    varios_value,
    apaurb_condition,
    apaurb_value,
    default_value,
    column_name="autoria_extra",
)

In [ ]:
df.loc[df["autoria"].str.startswith("APAURB"), "autoria"] = "APAURB"

In [ ]:
# with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
#     display(df["descricao"].str.split(".", n=1, expand=True, regex=False))

In [ ]:
author_year_condition = ~df["data"].isna()
author_year_value = df["descricao"].str.replace(r"^(.+)\s\(\d{4}\)\.?", "", regex=True)

author_condition = df["data"].isna()
author_value = df["descricao"].str.replace(r"^([\w &/]+)\.", "", regex=True)

default_value = pd.NA

df = df.case_when(
    author_year_condition,
    author_year_value,
    author_condition,
    author_value,
    default_value,
    column_name="descricao",
)

In [ ]:
# cols = ["autoria", "data", "descricao"]

# with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
#     display(df[cols])

**Quotation marks**

- « »
- “ ”

In [ ]:
# df["descricao"] = df["descricao"].str.replace("\"\"", "\"", regex=False)

In [ ]:
# "“" == "”"

In [ ]:
with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
    display(df[["id", "descricao"]])

In [ ]:
citation_double_ids = [1, 3, 28, 35]  # "" ""
citation_quotes_ids = [9, 15, 17, 19, 25, 47, 48]  # “ .”
citation_quotes_period_ids = [6]  # “ ”

citation_double_condition = df["id"].isin(citation_double_ids)
citation_quotes_condition = df["id"].isin(citation_quotes_ids)
citation_quotes_period_condition = df["id"].isin(citation_quotes_period_ids)

In [ ]:
citation_double_value_1 = df["descricao"].str.replace('""', "«", regex=False, n=1)
citation_quotes_value_1 = df["descricao"].str.replace("“", "«", regex=False, n=1)
citation_quotes_period_value_1 = df["descricao"].str.replace("“", "«", regex=False, n=1)

df = df.case_when(
    citation_double_condition,
    citation_double_value_1,
    citation_quotes_condition,
    citation_quotes_value_1,
    citation_quotes_period_condition,
    citation_quotes_period_value_1,
    df["descricao"],
    column_name="descricao",
)
# df.head()

In [ ]:
citation_double_value_2 = df["descricao"].str.replace('""', "»", regex=False, n=1)
citation_quotes_value_2 = df["descricao"].str.replace("”", "»", regex=False, n=1)
citation_quotes_period_value_2 = df["descricao"].str.replace(
    "”", ".»", regex=False, n=1
)

df = df.case_when(
    citation_double_condition,
    citation_double_value_2,
    citation_quotes_condition,
    citation_quotes_value_2,
    citation_quotes_period_condition,
    citation_quotes_period_value_2,
    df["descricao"],
    column_name="descricao",
)
# df.head()

In [ ]:
df["descricao"] = df["descricao"].str.strip()
df["name"] = df["name"].str.strip()
df["autoria"] = df["autoria"].str.strip()

## Output

In [ ]:
# indent = 2
indent = 4

df.to_json(OUTPUT_DATA, orient="records", force_ascii=False, indent=indent)

---